# CMSC 636 Data Visualization - Fall 2020

## The Last Dance: Legitimizing Event-Driven Analysis of the Air Jordan Sneaker Secondary Market

By: Sandeep Bansal, Atal Mutyala, Mansi Patil, Alex Pratte, Amanda Ross, Ben
Roytenberg



In [4]:
#@title Modules and classes required for this notebook to work (Run me!)
import pandas as pd
import urllib.parse
import requests
import random
import time
import datetime
import numpy as np
import matplotlib.pyplot as plot
%matplotlib inline
plot.rcParams['figure.figsize'] = [25, 10]
class ansi_colors:
    ERROR = "\033[91m"
    INFO  = "\u001b[35m"
    OK    = "\u001b[32m"
    RESET = "\033[0m"

### Step 1: Pick Your Sneakers

The data used for this market analysis tool is directly pulled from StockX's public API. You can search directly by running and interacting with the cell below, **OR** run the cell below to import your data from a .csv file.

**WARNING**: The StockX public API is _heavily_ rate limited. If you intend to query for a lot of different sneakers, checkpointing is easily handled when running locally, but _not as easily_ when running on Google Colaboratory. 

* If you get a rate limiting error when running locally, visit https://stockx.com, and solve the PerimeterX challenge before proceeding. 

* If running on Google Colaboratory, export this notebook, and reopen it separately from Google Drive. If your external IP changes from the Troubleshooting cell between notebooks, then you should be able to resolve it.

In [ ]:
#@title StockX Import Wizard (Run me!)
# Important things for later, mostly black magic
bypass = {'user-agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}
# calculate date away from September 30, 2020
september_30_2020 = datetime.date(2020, 9, 30)
def n_days_away(day_0, n):
   delta = datetime.timedelta(days = n)
   return day_0-delta
date_list = []
for i in range(0, 366):
    date_list.append(n_days_away(september_30_2020, i).strftime("%m-%d-%y"))
#cols = ['model_number', 'canonical_name', 'colorway', 'silhouette', 'gender', 'release_date', 'retail_price', 'product_url'] + list(map(str, list(range(0,366))[::-1]))
cols = ['model_number', 'canonical_name', 'colorway', 'silhouette', 'gender', 'release_date', 'retail_price', 'product_url'] + date_list[::-1]
# Initialize dataframe with our column names, will be accessible in later cells
df = pd.DataFrame(columns=cols)
# First prompt and gather of user input
print(f"{ansi_colors.INFO}Step 1: Pick Your Sneakers{ansi_colors.RESET}")
print("Type `exit` at any time to end execution of this cell and continue to Step 2.\n")
print(f"Enter some search terms to explore the {ansi_colors.OK}StockX{ansi_colors.RESET} product API:")
query = input()
print()
# Event loop
while query.strip().lower() != "exit":
  # URL encode the query
  urlenc = urllib.parse.quote(query)
  # GET our data from /browse/
  url_template = f'https://stockx.com/api/browse?_search={urlenc}&_tags=air%20jordan&productCategory=sneakers&order=DESC&page=1&dataType=product'
  resp = requests.get(url_template, headers=bypass)
  # Check if rate limited
  try:
    # This errors out if you get stuck at a checkpoint, since the body would be HTML
    browse_data = resp.json()
    page_lim = browse_data.get("Pagination").get("limit")
    num_matched = browse_data.get("Pagination").get("total")
    # Print out products found in search (if there are any)
    products = browse_data.get("Products")
    if len(products) != 0:
      print(f"{ansi_colors.OK}   Success!{ansi_colors.RESET} Showing {page_lim} of {num_matched} results:\n")
      for i in range(0, len(products)):
        name = products[i].get("title")
        if products[i].get("releaseDate"):
          rdate = "released " + products[i].get("releaseDate")[:10]
        else:
          rdate = ansi_colors.ERROR + "no release date" + ansi_colors.RESET
        print(f"{ansi_colors.INFO}{i+1}.{ansi_colors.RESET} {name} ({rdate})")
      print()
      # Query for a choice to be written to the dataframe
      print("Please input your choice, or `search` to return to search:")
      choice = input()
      invalid = True
      # Check bounds on choices
      while invalid:
        try:
          num = int(choice)
          if num > len(products) or num < 1:
            print(f"{ansi_colors.ERROR}ERROR:{ansi_colors.RESET} Invalid selection. Please choose a number between 1 and {len(products)}")
            choice = input()
          else:
            invalid = False
        except:
          if choice == "search":
            invalid = False
          else:
            print(f"{ansi_colors.ERROR}ERROR:{ansi_colors.RESET} Invalid selection. Please choose a number between 1 and {len(products)}, or `search` to return to search")
            choice = input()
      if choice.strip().lower() == "search":
        pass
      else:
        choice = int(choice)
        uuid = products[choice-1].get('uuid')
        # Need to patch in date selection later...
        market_history_url = f'https://stockx.com/api/products/{uuid}/chart?start_date=2019-10-01&end_date=2020-10-01&intervals=366&currency=USD&country=US'
        market_data = requests.get(market_history_url, headers=bypass).json().get('series')[0].get('data')
        row = [products[choice-1].get("category"), 
                products[choice-1].get('name'),
                products[choice-1].get('colorway'),
                products[choice-1].get('shoe'),
                products[choice-1].get('gender'), 
                products[choice-1].get('releaseDate'),
                products[choice-1].get('retailPrice'),
                'https://stockx.com/' + products[choice-1].get('urlKey')] + market_data
        df = df.append(pd.Series(row, index=df.columns), ignore_index=True)
        print()
        print(f"{ansi_colors.OK}It's in!{ansi_colors.INFO} {products[choice-1].get('title')}{ansi_colors.RESET} has been added to the dataframe.\n")
    else:
      print(f"No search results found for {query}. Returning to search...")
  except:
    print(f"{ansi_colors.ERROR}ERROR:{ansi_colors.RESET} rate limited. Visit https://stockx.com to solve the challenge, and try again...\n")
    time.sleep(3)
  print(f"Enter some search terms to explore the {ansi_colors.OK}StockX{ansi_colors.RESET} product API:")
  query = input()
  print()
print(f"{ansi_colors.INFO}Thank you for completing this step. Here is a sneak peek at your dataframe:{ansi_colors.RESET}")
df.to_csv('your_sneaker_data.csv') 
df.head()

Step 1: Pick Your Sneakers
Type `exit` at any time to end execution of this cell and continue to Step 2.

Enter some search terms to explore the StockX product API:
3

   Success! Showing 40 of 1997 results:

1. Jordan 3 Retro Racer Blue (released 2021-07-10)
2. Jordan 3 Retro Georgetown (2021) (released 2021-03-20)
3. Jordan 3 Retro Racer Blue (GS) (released 2021-07-10)
4. Jordan 4 Retro SP 30th Anniversary Union Desert Moss (released 2021-06-24)
5. Jordan 3 Retro Cool Grey (2021) (released 2021-02-20)
6. Jordan 3 Retro A Ma Maniére (W) (released 2021-06-10)
7. Jordan 3 Retro Fragment (released 2020-09-17)
8. Jordan 3 Retro Rust Pink (W) (released 2021-05-28)
9. Jordan 1 Retro High Shattered Backboard 3.0 (released 2019-10-26)
10. Jordan 36 SE Luka Doncic PE (released 2021-09-04)
11. Jordan 3 Retro SE Fire Red Denim (2020) (released 2020-08-27)
12. Jordan 3 Retro SE Unite Fire Red (released 2020-02-15)
13. Jordan 3 Retro Varsity Royal Cement (released 2020-10-10)
14. Jordan 3 Retro Ge

In [ ]:
#@title Import Data (Run me if you didn't use the wizard)
filename = 'your_sneaker_data.csv' #@param {type:"string"}
df = pd.read_csv(filename) 
df.head()

In [ ]:
#@title Troubleshooting (See Your External IP Address)
print(requests.get('http://icanhazip.com').text)

### Step 2: Process Your Data

Here are a handful of cells that you can run to further process your data. Whichever cells you run are your choice, and overwrites the initial `df` dataframe, so be careful in what you choose to run. Currently supported operations:

- Timeline binning

- Event highlighting

In [ ]:
#@title Processing Options (Run me!)
# User configuration
rolling_avg = 30 #@param {type:"slider", min:1, max:30, step:1}
start_date_input = '2020-04-19' #@param {type:"date"}
formatted_start_date = datetime.datetime.strptime(start_date_input, '%Y-%m-%d').strftime("%B-%d-%Y")
end_date_input = '2020-05-17' #@param {type:"date"}
formatted_end_date = datetime.datetime.strptime(end_date_input, '%Y-%m-%d').strftime("%B-%d-%Y")
binning = 'WEEK' #@param [ 'MONTH', 'WEEK']


In [ ]:
#@title Data Processing (Run me!)
# Just the prices, no metadata
df2 = df.drop(df.columns[0:9], axis=1)

# Create month binning
if binning == 'MONTH':
  # Quarter 1
  january_bin = np.transpose(df[[col for col in df if col.startswith('01')]].T.to_numpy())
  february_bin = np.transpose(df[[col for col in df if col.startswith('02')]].T.to_numpy())
  march_bin = np.transpose(df[[col for col in df if col.startswith('03')]].T.to_numpy())
  # Quarter 2 
  april_bin = np.transpose(df[[col for col in df if col.startswith('04')]].T.to_numpy())
  may_bin = np.transpose(df[[col for col in df if col.startswith('05')]].T.to_numpy())
  june_bin = np.transpose(df[[col for col in df if col.startswith('06')]].T.to_numpy())
  #  Quarter 3 
  july_bin = np.transpose(df[[col for col in df if col.startswith('07')]].T.to_numpy())
  august_bin = np.transpose(df[[col for col in df if col.startswith('08')]].T.to_numpy())
  september_bin = np.transpose(df[[col for col in df if col.startswith('09')]].T.to_numpy())
  # Quarter 4
  october_bin = np.transpose(df[[col for col in df if col.startswith('10')]].T.to_numpy())
  november_bin = np.transpose(df[[col for col in df if col.startswith('11')]].T.to_numpy())
  december_bin = np.transpose(df[[col for col in df if col.startswith('12')]].T.to_numpy())
  # Month names
  months=[ "Oct 2019", "Nov 2019", "Dec 2019","Jan 2020", "Feb 2020", "Mar 2020", "Apr 2020", "May 2020", "Jun 2020", "Jul 2020","Aug 2020", "Sep 2020"]
  # Bin array
  month_bin_array = [october_bin,november_bin,december_bin,january_bin,february_bin,march_bin,april_bin,may_bin,june_bin,july_bin,august_bin,september_bin]

# Create week binning
elif binning == 'WEEK':
  week_bin_array = []
  day_count = 0
  week_count = -1
  for col in df2:
    if day_count % 7 == 0:
        week_count+=1
        week_bin_array.append([])
    week_bin_array[week_count].append(df2[col].T.to_numpy())
    day_count +=1  

  for week_num in range(0, len(week_bin_array)):
    week_bin_array[week_num] = np.transpose(week_bin_array[week_num])


new_columns = df['model_number'].str.cat(df['canonical_name'],sep=" ")
new_columns = new_columns.str.cat(df['colorway'],sep=" ")

df2=df2.T
df2.columns=new_columns.T.to_numpy()
df2.index = pd.to_datetime(df2.index)
df2.head()

df2 = df2.rolling(rolling_avg).mean() #seven day rolling average


### Step 3: Visualize!


In [ ]:
#@title Visualization 1: Time Series Composite Line Chart w/ Event Highlighting (Run me!)
df2.plot()
top = int(plot.gca().get_ylim()[1])
plot.axvline(pd.to_datetime(start_date_input), color='k', linestyle='--', lw=2,label='Documentary Start Date ('+formatted_start_date+')')
plot.axvline(pd.to_datetime(end_date_input), color='k', linestyle='--', lw=2,label='Documentary End Date (' + formatted_end_date + ')')
plot.text(x='2020-04-19',y=0,s='Start Date',horizontalalignment='right',fontweight='bold',rotation=45,fontsize='large')
plot.text(x='2020-05-17',y=0,s='End Date',horizontalalignment='left',fontweight='bold',rotation=45,fontsize='large')
fig=plot.gcf()
fig.set_size_inches(25,10)

_=plot.legend(bbox_to_anchor=(1.02,1),title='Shoe Name',title_fontsize='x-large',loc="upper left",labelspacing=1.6)
_=plot.ylabel('Average Resale Price ($)',fontsize=18)
_=plot.xlabel('Month of Year',fontsize=18)
_=plot.grid(True)
_=plot.title('Average Resale Price of User Selected Sneakers',fontsize=26)


In [ ]:
#@title Visualization 2: Time Series Box and Whisker Plot (Run me!)
if binning == 'MONTH':
  for shoe_select in range(0, len(df["model_number"])):
    selected = []
    for i in month_bin_array:
      selected.append(i[shoe_select])
    plt.boxplot(selected)
    plt.xticks(range(1, len(month_bin_array)+1), months, rotation='vertical')
    plt.title(df["model_number"][shoe_select] + " " + df["canonical_name"][shoe_select] + " Sales Per Month")
    plt.show()

elif binning == 'WEEK':
  for shoe_select in range(0, len(df["model_number"])):
    selected = []
    for i in week_bin_array:
      selected.append(i[shoe_select])
    plot.boxplot(selected)
    plot.title(df["model_number"][shoe_select] + " " + df["canonical_name"][shoe_select] + " Sales Per Week Since Oct 2019")
    _=plot.ylabel('Average Resale Price ($)',fontsize=18)
    _=plot.xlabel('Week of Year',fontsize=18) 
    plot.show()